# Objetivo

En esta práctica, se debe desarrollar un código capaz de reconocer rostros en tiempo real a través de la cámara web. Los objetivos específicos son los siguientes:    

## Entrenamiento del Modelo:
* Crear un modelo de reconocimiento facial.
* Entrenar el modelo utilizando imágenes de cinco personas distintas.
* Asegurarse de que todas las imágenes utilizadas tengan una resolución mínima de 100 x 100 píxeles y una resolución máxima de 150 x 150 píxeles.


## Pruebas con Tres Modelos Diferentes:
Evaluar el rendimiento del modelo utilizando tres algoritmos distintos:
* EigenFaceRecognizer
* FisherFaceRecognizer
* LBPHFaceRecognizer

________________________

# Desarrollo

Se importan las librerías necesarias.

In [1]:
import cv2 as cv 
import numpy as np 
import os

El siguiente código se encarga de tomar capturas de los rostros que el sistema detecte mediante la cámara web con la ayuda de un archivo ya entrenado de tipo HaarCascade para identificar los rostros:

In [ ]:
usuario = 'axel'

In [3]:
cap = cv.VideoCapture(0)
i = 0
while True:
    ret, frame = cap.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    rostros = rostro.detectMultiScale(gray, 1.3, 5)
    for(x, y, w, h) in rostros:
        #frame = cv.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        frame2 = frame[y:y+h, x:x+w]
        #cv.imshow('rostros', frame)
        frame2 = cv.resize(frame2, (100, 100))
        cv.imshow('rostros', frame2)
        cv.imwrite('./saved_img/' + usuario + '/' + usuario + str(i) + '.png', frame2)
    i = i + 1
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

En el siguiente código se inicializan las variables necesarias para ubicar y almacenar tanto las imágenes como las etiquetas respectivas para cada rostro.

In [2]:
dataSet = './caras'
faces  = os.listdir(dataSet)
print(faces)

labels = []
facesData = []
label = 0 
for face in faces:
    facePath = dataSet+'/'+face
    for faceName in os.listdir(facePath):
        labels.append(label)
        facesData.append(cv.imread(facePath+'/'+faceName,0))
    label = label + 1
print(np.count_nonzero(np.array(labels)==0)) 

['axel', 'brenda', 'dante', 'martin', 'rodrigo']
453


## Eigenface 

Este bloque de código se encarga de crear un xml de entrenamiento para detectar los rostros mediante el modelo Eigenface.

In [2]:
faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('./modelos/CarasEigenface.xml')

['axel', 'brenda', 'dante', 'martin', 'rodrigo']
453


## Fisherfaces

Este bloque de código se encarga de crear un xml de entrenamiento para detectar los rostros mediante el modelo Fisherfaces.

In [3]:
faceRecognizer = cv.face.FisherFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('./modelos/CarasFisherface.xml')

## LBPH

Este bloque de código se encarga de crear un xml de entrenamiento para detectar los rostros mediante el modelo LBPH.

In [4]:
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.train(facesData, np.array(labels))
faceRecognizer.write('./modelos/CarasLBPH.xml')

___________

# Pruebas

Para las pruebas de funcionamiento se desarrollaron tres bloques de código para trabajar con cada uno de los modelos:

## Eigenface 

In [5]:
faceRecognizer = cv.face.EigenFaceRecognizer_create()
faceRecognizer.read('./modelos/CarasEigenface.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier('./haarcascade/haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        #cv.putText(frame, '{}'.format(result), (x,y-20), 1,3.3, (255,255,0), 1, cv.LINE_AA)
        if result[1] > 2800:
            # el arreglo result es algo así: [etiqueta: 1, valor: 3000]
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
    cv.imshow('Caras Eigenface', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()


## Fisherfaces

In [6]:
faceRecognizer = cv.face.FisherFaceRecognizer_create()
faceRecognizer.read('./modelos/CarasFisherface.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier('./haarcascade/haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        # cv.putText(frame, '{}'.format(result), (x,y-20), 1,3.3, (255,255,0), 1, cv.LINE_AA)
        if result[1] > 2:
            # el arreglo result es algo así: [etiqueta: 1, valor: 3000]
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
    cv.imshow('Caras Fisherface', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()


## LBPH

In [7]:
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.read('./modelos/CarasLBPH.xml')

cap = cv.VideoCapture(0)
rostro = cv.CascadeClassifier('./haarcascade/haarcascade_frontalface_alt.xml')
while True:
    ret, frame = cap.read()
    if ret == False: break
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    for(x, y, w, h) in rostros:
        frame2 = cpGray[y:y+h, x:x+w]
        frame2 = cv.resize(frame2,  (100,100), interpolation=cv.INTER_CUBIC)
        result = faceRecognizer.predict(frame2)
        # cv.putText(frame, '{}'.format(result), (x,y-20), 1,3.3, (255,255,0), 1, cv.LINE_AA)
        if result[1] > 80:
            # el arreglo result es algo así: [etiqueta: 1, valor: 3000]
            cv.putText(frame,'{}'.format(faces[result[0]]),(x,y-25),2,1.1,(0,255,0),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,255,0),2)
        else:
            cv.putText(frame,'Desconocido',(x,y-20),2,0.8,(0,0,255),1,cv.LINE_AA)
            cv.rectangle(frame, (x,y),(x+w,y+h),(0,0,255),2)
    cv.imshow('Caras LBPH', frame)
    k = cv.waitKey(1)
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()
